In [2]:
import yfinance as yf
import pandas as pd

# 输入公司的股票代码
ticker_symbol = "AAPL"  # 以苹果公司为例

# # 获取公司的财务报表数据
# company = yf.Ticker(ticker_symbol)

# # 获取利润表数据
# income_statement = company.financials
# # 获取资产负债表数据
# balance_sheet = company.balance_sheet
# # 获取现金流量表数据
# cash_flow = company.cashflow

# # 写入csv文件
# income_statement.to_csv("income_statement.csv")
# balance_sheet.to_csv("balance_sheet.csv")
# cash_flow.to_csv("cash_flow.csv")

In [3]:
income_statement = pd.read_csv("income_statement.csv")
balance_sheet = pd.read_csv("balance_sheet.csv")
cash_flow = pd.read_csv("cash_flow.csv")

In [4]:
def transpose(df):
    df = df.T   # 转置
    df.columns = df.iloc[0] # 将第一行设置为列名
    df = df.iloc[1:]    # 删除第一行
    df = df.iloc[::-1]  # 倒序
    return df

In [5]:
income_statement = transpose(income_statement)
balance_sheet = transpose(balance_sheet)
cash_flow = transpose(cash_flow)

In [6]:
income_statement

Unnamed: 0,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,Net Interest Income,Interest Expense,...,Interest Expense Non Operating,Interest Income Non Operating,Operating Income,Operating Expense,Research And Development,Selling General And Administration,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
2020-09-30,0.0,0.144282,77344000000.0,57411000000.0,11056000000.0,169559000000.0,77344000000.0,66288000000.0,890000000.0,2873000000.0,...,2873000000.0,3763000000.0,66288000000.0,38668000000.0,18752000000.0,19916000000.0,104956000000.0,169559000000.0,274515000000.0,274515000000.0
2021-09-30,0.0,0.133023,120233000000.0,94680000000.0,11284000000.0,212981000000.0,120233000000.0,108949000000.0,198000000.0,2645000000.0,...,2645000000.0,2843000000.0,108949000000.0,43887000000.0,21914000000.0,21973000000.0,152836000000.0,212981000000.0,365817000000.0,365817000000.0
2022-09-30,0.0,0.162045,130541000000.0,99803000000.0,11104000000.0,223546000000.0,130541000000.0,119437000000.0,-106000000.0,2931000000.0,...,2931000000.0,2825000000.0,119437000000.0,51345000000.0,26251000000.0,25094000000.0,170782000000.0,223546000000.0,394328000000.0,394328000000.0
2023-09-30,0.0,0.147192,125820000000.0,96995000000.0,11519000000.0,214137000000.0,125820000000.0,114301000000.0,NaN,NaN,...,NaN,NaN,114301000000.0,54847000000.0,29915000000.0,24932000000.0,169148000000.0,214137000000.0,383285000000.0,383285000000.0


In [7]:
df = pd.DataFrame()
df['毛利率'] = (income_statement['Gross Profit']/income_statement['Total Revenue']).astype(float)
df['净利率'] = (income_statement['Net Income']/income_statement['Total Revenue']).astype(float)
df['资产回报率'] = (income_statement['Net Income']/balance_sheet['Total Assets']).astype(float)

df['收入变化'] = income_statement['Total Revenue'].diff()
df['净利润变化'] = income_statement['Net Income'].diff()
df['收入变化率'] = income_statement['Total Revenue'].pct_change()
df['净利润变化率'] = income_statement['Net Income'].pct_change()

In [8]:
df

,毛利率,净利率,资产回报率,收入变化,净利润变化,收入变化率,净利润变化率
2020-09-30,0.382332,0.209136,0.177256,NaN,NaN,NaN,NaN
2021-09-30,0.417794,0.258818,0.269742,91302000000.0,37269000000.0,0.332594,0.649161
2022-09-30,0.433096,0.253096,0.282924,28511000000.0,5123000000.0,0.077938,0.054109
2023-09-30,0.441311,0.253062,0.275098,-11043000000.0,-2808000000.0,-0.028005,-0.028135


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

tickvals=[2020,2021,2022,2023]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.04,
                    specs=[[{"secondary_y": True, "type": "xy"}], 
                           [{"type": "table"}]])

fig.add_trace(go.Bar(
       x=tickvals,
       y=income_statement['Total Revenue'],
       name="总收入",
       marker=dict(color="rgb(80, 162, 174)")),
       row=1, col=1, secondary_y=False
       )

fig.add_trace(go.Bar(
       x=tickvals,
       y=income_statement['Gross Profit'],
       name="毛利润",
       marker=dict(color="rgb(120, 144, 154)")),
       row=1, col=1, secondary_y=False
       )

fig.add_trace(go.Bar(
       x=tickvals,
       y=income_statement['Net Income'],
       name="净利润",
       marker=dict(color="rgb(123, 164, 151)")),
       row=1, col=1, secondary_y=False
       )

fig.add_trace(go.Scatter(
       x=tickvals,
       y=df['毛利率'],
       name="毛利率",
       hovertemplate = '%{y:.2%}',
       line=dict(color="rgb(97, 114, 124)")),
       row=1, col=1, secondary_y=True
       )

fig.add_trace(go.Scatter(
       x=tickvals,
       y=df['净利率'],
       name="净利率",
       hovertemplate = '%{y:.2%}',
       line=dict(color="rgb(80, 134, 121)")),
       row=1, col=1, secondary_y=True
       )

table_values = [tickvals,
                income_statement['Total Revenue'] / 1000000,
                income_statement['Gross Profit'] /1000000,
                (df['毛利率']*100).round(2),
                income_statement['Net Income'] / 1000000,
                (df['净利率']*100).round(2)]
                

fig.add_trace(go.Table(
       header=dict(
              values=["年份","总收入(M$)","毛利润(M$)","毛利率(%)","净利润(M$)","净利率(%)"],
              font=dict(size=12, color='white'),
              align="center",
              fill_color='grey',
              line_color='darkslategray',
              ),
       cells=dict(
              values=table_values,
              font = dict(color = 'darkslategray', size = 11),
              align = "center",
              fill_color=[['white','lightgrey']*4],
              line_color='darkslategray',
              )
       ),
       row=2, col=1
       )

fig.update_layout(
       height=800,
       width=800,
       plot_bgcolor="white",
       title_text="毛利润和净利润",
       hovermode="x unified",
       legend=dict(
           orientation="h",
           yanchor="bottom",
           y=1.02,
           xanchor="right",
           x=0.9
           )
       )

fig.update_xaxes(tickvals=tickvals, ticktext=tickvals, showgrid=False, row=1, col=1)
fig.update_yaxes(title_text='利润($)', gridcolor='rgb(204, 204, 204)',zerolinecolor="rgb(204, 204, 204)", secondary_y=False, row=1, col=1)

fig.show()

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

tickvals=[2020,2021,2022,2023]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(
       x=tickvals,
       y=df['收入变化'],
       name="收入变化",
       marker=dict(color="rgb(49, 122, 134)")),
       secondary_y=False
       )

fig.add_trace(go.Bar(
       x=tickvals,
       y=df['净利润变化'],
       name="净利润变化",
       marker=dict(color="rgb(93, 134, 121)")),
       secondary_y=False
       )

fig.add_trace(go.Scatter(
       x=tickvals,
       y=df['收入变化率'],
       name="收入变化率",
       hovertemplate = '%{y:.2%}',
       line=dict(color="rgb(30, 110, 120)")),
       secondary_y=True
       )

fig.add_trace(go.Scatter(
       x=tickvals,
       y=df['净利润变化率'],
       name="净利润变化率",
       hovertemplate = '%{y:.2%}',
       line=dict(color="rgb(93, 134, 121)")),
       secondary_y=True
       )

fig.update_layout(
       height=500,
       width=800,
       plot_bgcolor="white",
       title_text="收入和净利润变化",
       hovermode="x unified",
       legend=dict(
           orientation="h",
           yanchor="bottom",
           y=1.02,
           xanchor="right",
           x=0.9
           )
       )

fig.update_xaxes(tickvals=tickvals, ticktext=tickvals, showgrid=False)
fig.update_yaxes(title_text='利润($)', gridcolor='rgb(204, 204, 204)',zerolinecolor="rgb(204, 204, 204)", secondary_y=False)
fig.show()

In [11]:
balance_sheet

Unnamed: 0,Treasury Shares Number,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Common Stock Equity,...,Other Current Assets,Inventory,Receivables,Other Receivables,Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Equivalents,Cash Financial
2020-09-30,NaN,16976763000.0,16976763000.0,74420000000.0,112436000000.0,65339000000.0,177775000000.0,38321000000.0,65339000000.0,65339000000.0,...,11264000000.0,4061000000.0,37445000000.0,21325000000.0,16120000000.0,90943000000.0,52927000000.0,38016000000.0,20243000000.0,17773000000.0
2021-09-30,NaN,16426786000.0,16426786000.0,89779000000.0,124719000000.0,63090000000.0,187809000000.0,9355000000.0,63090000000.0,63090000000.0,...,14111000000.0,6580000000.0,51506000000.0,25228000000.0,26278000000.0,62639000000.0,27699000000.0,34940000000.0,17635000000.0,17305000000.0
2022-09-30,NaN,15943425000.0,15943425000.0,96423000000.0,120069000000.0,50672000000.0,170741000000.0,-18577000000.0,50672000000.0,50672000000.0,...,21223000000.0,4946000000.0,60932000000.0,32748000000.0,28184000000.0,48304000000.0,24658000000.0,23646000000.0,5100000000.0,18546000000.0
2023-09-30,0.0,15550061000.0,15550061000.0,81123000000.0,111088000000.0,62146000000.0,173234000000.0,-1742000000.0,62146000000.0,62146000000.0,...,14695000000.0,6331000000.0,60985000000.0,31477000000.0,29508000000.0,61555000000.0,31590000000.0,29965000000.0,NaN,NaN


In [12]:
df['债务比率'] = (balance_sheet['Total Liabilities Net Minority Interest']/balance_sheet['Total Assets']).astype(float)
df['流动比率'] = (balance_sheet['Current Assets']/balance_sheet['Current Liabilities']).astype(float)

In [13]:
df

,毛利率,净利率,资产回报率,收入变化,净利润变化,收入变化率,净利润变化率,债务比率,流动比率
2020-09-30,0.382332,0.209136,0.177256,NaN,NaN,NaN,NaN,0.798267,1.363604
2021-09-30,0.417794,0.258818,0.269742,91302000000.0,37269000000.0,0.332594,0.649161,0.820257,1.074553
2022-09-30,0.433096,0.253096,0.282924,28511000000.0,5123000000.0,0.077938,0.054109,0.856354,0.879356
2023-09-30,0.441311,0.253062,0.275098,-11043000000.0,-2808000000.0,-0.028005,-0.028135,0.823741,0.988012


In [14]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.07,
                    subplot_titles=("债务比率", "流动比率"),
                    specs=[[{"secondary_y": True, "type": "xy"}], 
                           [{"secondary_y": True, "type": "xy"}]])

fig.add_trace(go.Bar(
       x=tickvals,
       y=balance_sheet['Total Assets'],
       name="资产",
       marker=dict(color="rgb(80, 162, 174)")),
       row=1, col=1, secondary_y=False
       )

fig.add_trace(go.Bar(
       x=tickvals,
       y=balance_sheet['Total Liabilities Net Minority Interest'],
       name="负债",
       marker=dict(color="rgb(120, 144, 154)")),
       row=1, col=1, secondary_y=False
       )

fig.add_trace(go.Scatter(
       x=tickvals,
       y=df['债务比率'],
       name="债务比率",
       hovertemplate = '%{y:.2%}',
       line=dict(color="rgb(34, 186, 187)")),
       row=1, col=1, secondary_y=True
       )

fig.add_trace(go.Bar(
       x=tickvals,
       y=balance_sheet['Current Assets'],
       name="流动资产",
       marker=dict(color="rgb(0, 171, 189)")),
       row=2, col=1, secondary_y=False
       )

fig.add_trace(go.Bar(
       x=tickvals,
       y=balance_sheet['Current Liabilities'],
       name="流动负债",
       marker=dict(color="rgb(161, 199, 224)")),
       row=2, col=1, secondary_y=False
       )

fig.add_trace(go.Scatter(
       x=tickvals,
       y=df['流动比率'],
       name="流动比率",
       hovertemplate = '%{y:.2%}',
       line=dict(color="rgb(0, 153, 221)")),
       row=2, col=1, secondary_y=True
       )

fig.update_layout(
       height=800,
       width=800,
       plot_bgcolor="white",
       title_text="债务比率和流动比率",
       hovermode="x unified",
       legend=dict(
           orientation="h",
           yanchor="bottom",
           y=1.02,
           xanchor="right",
           x=0.9
           )
       )

fig.update_xaxes(tickvals=tickvals, ticktext=tickvals, showgrid=False, row=1, col=1)
fig.update_yaxes(title_text='$', gridcolor='rgb(204, 204, 204)',zerolinecolor="rgb(204, 204, 204)", secondary_y=False, row=1, col=1)
fig.update_yaxes(gridcolor='rgb(204, 204, 204)',zerolinecolor="rgb(204, 204, 204)", secondary_y=True, row=1, col=1)

fig.update_xaxes(tickvals=tickvals, ticktext=tickvals, showgrid=False, row=2, col=1)
fig.update_yaxes(title_text='$', gridcolor='rgb(204, 204, 204)',zerolinecolor="rgb(204, 204, 204)", secondary_y=False, row=2, col=1)
fig.update_yaxes(gridcolor='rgb(204, 204, 204)',zerolinecolor="rgb(204, 204, 204)", secondary_y=True, row=2, col=1)

fig.show()

In [15]:
cash_flow.columns

Index(['Free Cash Flow', 'Repurchase Of Capital Stock', 'Repayment Of Debt',
       'Issuance Of Debt', 'Issuance Of Capital Stock', 'Capital Expenditure',
       'Interest Paid Supplemental Data', 'Income Tax Paid Supplemental Data',
       'End Cash Position', 'Beginning Cash Position', 'Changes In Cash',
       'Financing Cash Flow', 'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Cash Dividends Paid',
       'Common Stock Dividend Paid', 'Net Common Stock Issuance',
       'Common Stock Payments', 'Common Stock Issuance',
       'Net Issuance Payments Of Debt', 'Net Short Term Debt Issuance',
       'Short Term Debt Payments', 'Short Term Debt Issuance',
       'Net Long Term Debt Issuance', 'Long Term Debt Payments',
       'Long Term Debt Issuance', 'Investing Cash Flow',
       'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Net Investment Purchase And Sale',
       'Sale Of Investment', 'Purchase 

In [16]:
cash_flow[['Operating Cash Flow','Investing Cash Flow','Financing Cash Flow','Free Cash Flow']]

Unnamed: 0,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow
2020-09-30,80674000000.0,-4289000000.0,-86820000000.0,73365000000.0
2021-09-30,104038000000.0,-14545000000.0,-93353000000.0,92953000000.0
2022-09-30,122151000000.0,-22354000000.0,-110749000000.0,111443000000.0
2023-09-30,110543000000.0,3705000000.0,-108488000000.0,99584000000.0


In [17]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=tickvals,
    y=cash_flow['Operating Cash Flow'],
    name="经营现金流",
    marker=dict(color="rgb(80, 162, 174)"))
    )

fig.add_trace(go.Bar(
    x=tickvals,
    y=cash_flow['Investing Cash Flow'],
    name="投资现金流",
    marker=dict(color="rgb(120, 144, 154)"))
    )

fig.add_trace(go.Bar(
    x=tickvals,
    y=cash_flow['Financing Cash Flow'],
    name="筹资现金流",
    marker=dict(color="rgb(123, 164, 151)"))
    )

fig.add_trace(go.Bar(
    x=tickvals,
    y=cash_flow['Free Cash Flow'],
    name="自由现金流",
    marker=dict(color="rgb(97, 114, 124)"))
    )

fig.update_layout(
    height=500,
    width=800,
    plot_bgcolor="white",
    title_text="现金流",
    hovermode="x unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.9
        )
    )

fig.update_xaxes(tickvals=tickvals, ticktext=tickvals, showgrid=False)
fig.update_yaxes(title_text='$', gridcolor='rgb(204, 204, 204)',zerolinecolor="rgb(204, 204, 204)")

fig.show()

In [18]:
import numpy as np
import pandas as pd

def calculate_dcf(cash_flows, discount_rate):
    # 计算每期的现金流的折现值
    present_values = [cf / (1 + discount_rate) ** i for i, cf in enumerate(cash_flows, start=1)]

    # 计算终端值的折现值
    terminal_value = cash_flows[-1] / discount_rate

    # 计算公司价值
    company_value = sum(present_values) + terminal_value

    return company_value

# 示例数据
cash_flows = [100, 150, 200, 250, 300]
discount_rate = 0.1

# 计算DCF
result = calculate_dcf(cash_flows, discount_rate)
print(f"公司价值为: {result}")


公司价值为: 3722.168753997181


In [19]:
free_cash_flow = cash_flow['Free Cash Flow'].values[-1]
cash_flows = [free_cash_flow * (1 + 0.1) ** i for i in range(1, 6)]
calculate_dcf(cash_flows, 0.1)

2101730278400.0005

In [20]:
cash_flows

[109542400000.00002,
 120496640000.00002,
 132546304000.00005,
 145800934400.00003,
 160381027840.00006]